In [1]:
#import modules
import pandas as pd

In [ ]:
cc_arson_df = pd.read_csv("../data/Crimes_and_Clearances_with_Arson-1985-2023.csv")

/tmp/ipykernel_105711/1081854286.py:1: DtypeWarning: Columns (21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/Crimes_and_Clearances_with_Arson-1985-2023.csv")


The message above indicates that some of the columns have mixed data type. We can resolve this below:
- First check which columns have non-numeric data types.
- Check which columns have mixed data types. 

In [ ]:
## Columns with Non numeric dtypes
non_numeric_cols = cc_arson_df.select_dtypes(include=['object']).columns
print(non_numeric_cols)

Index(['County', 'NCICCode', 'TotalStructural_sum', 'TotalMobile_sum',
       'TotalOther_sum', 'GrandTotal_sum', 'GrandTotClr_sum'],
      dtype='object')


In [6]:
def mixed_type_columns(df: pd.DataFrame) -> list:
    """
    Look for columns with mixed types

    Parameters: 
    df(pd.DataFrame)

    Returns: 
    list: A list of columes with mixed dtypes
    """
    mixed_columns = []
    for column in df.columns:
        types_in_column = df[column].map(type).unique()
        if len(types_in_column) > 1:
            mixed_columns.append(column)
    return mixed_columns   

In [ ]:
# The columns with mixed dtypes
mixed_columns = mixed_type_columns(cc_arson_df)
print(mixed_columns)

['TotalStructural_sum', 'TotalMobile_sum', 'TotalOther_sum', 'GrandTotal_sum', 'GrandTotClr_sum']


In [ ]:
# Make a copy of df
cc_arson_df_copy = cc_arson_df.copy()


In [ ]:
# Resolve the issue with mixed dtypes

cc_arson_df_copy[mixed_columns] = cc_arson_df_copy[mixed_columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
# No mixed types in the copy of the dataframe.
mixed_columns = mixed_type_columns(cc_arson_df_copy)
print(mixed_columns == [])

True
